# Time-series approach

In [37]:
import seaborn as sns

from sklearn.model_selection import train_test_split

from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from tslearn.preprocessing import TimeSeriesScalerMinMax,TimeSeriesScalerMeanVariance

from tqdm import tqdm
import numpy as np

from IPython.display import display

import sys
sys.path.append('../src')
from utils import *
sys.path.remove('../src')

In [90]:
df_samples = pd.read_csv('../data/data.csv', index_col='Unnamed: 0')
display(df_samples.head(4))

df_targets = pd.read_csv('../data/target_train.csv')
df_targets['labels'] = df_targets['target'].apply(lambda x: 0 if x == 'OD' else 1)
display(df_targets)

X, y =[], []
for id in tqdm(df_targets['id']):
    sample = np.vstack([df_samples[df_samples['id'] == id]['ch0'].to_numpy(),
                        df_samples[df_samples['id'] == id]['ch1'].to_numpy(),
                        df_samples[df_samples['id'] == id]['ch2'].to_numpy()])
    target = df_targets[df_targets['id']==id]['labels']
    X.append(sample)
    y.append(target)
X = np.array(X)
y = np.array(y)
y = np.reshape(y, (y.shape[0], ))

,id,time,ch0,ch1,ch2
0,1_1,0,372.271894,350.219643,408.527289
1,1_1,1,373.393195,349.845876,411.143658
2,1_1,2,371.898127,350.219643,412.638726
3,1_1,3,371.898127,350.967177,413.386260


,id,target,labels
0,111_1,OD,0
1,103_1,OD,0
2,34_2,ID,1
3,78_1,OD,0
4,71_2,OD,0
...,...,...,...
95,136_1,OD,0
96,183_1,ID,1
97,26_2,OD,0
98,6_2,OD,0


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:11<00:00,  8.61it/s]


In [115]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
knn = KNeighborsTimeSeriesClassifier(n_neighbors=2, metric="dtw")

knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

from sklearn.metrics import f1_score, accuracy_score

print(accuracy_score(y_test, y_pred))
f1 = f1_score(y_test, y_pred, average='weighted')
print(f1)

0.85
0.832258064516129


In [118]:
knn.to_pickle('../models/knn_ts.pkl')